In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#instalar doctr c/pytorch
!pip install python-doctr[torch]
!pip install --user tensorflow-addons==0.8.3
!pip install --user tensorflow==2.2.0-rc3
!sudo apt-get install fonts-freefont-ttf -y

In [ ]:
!pip install pdf2image
!apt-get install poppler-utils 

In [ ]:
!pip install pypdfium2==1.11.0

In [1]:
#Aqui se inicia el codigo de procesamiento de texto como tal
%matplotlib inline
import os

# Let's pick the desired backend
# os.environ['USE_TF'] = '1'
os.environ['USE_TORCH'] = '1'

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import os
from tqdm.auto import tqdm


import tensorflow as tf #Usar con los modelos 

#Se importan las funciones de la libreria docTR
from doctr.io import DocumentFile
#from doctr.models import ocr_predictor
from doctr.models import resnet18
from doctr.models import ocr_predictor

from pdf2image import convert_from_path

In [2]:
predictor = ocr_predictor(pretrained=True).cuda()

  0%|          | 0/101971449 [00:00<?, ?it/s]

  0%|          | 0/63286381 [00:00<?, ?it/s]

In [3]:
#Funcion para el procesamiento del formato y extraccion de
#las palabras y coordenadas contenidas en el JSON
def text_capture(json_export):

  #inicializamos las listas para captura de palabras y coordenadas
  words = []
  boxes = []

  #Proceso de iteracion para el acceso a niveles y procesar la extraccion del texto
  #(Ver Texto **Estrucura del JSON**)

  for fields in json_export['pages']:

    for subfields in fields['blocks']:

      for subfields2 in subfields['lines']:

        for subfields3 in subfields2['words']:
          
          words.append(subfields3['value'])
          bboxes=[int(x*1000) for x in list(subfields3['geometry'][0] + subfields3['geometry'][1])]
          boxes.append(bboxes)

  return words, boxes

In [4]:
def pdfToImage(url_init,url_final,file_name):
  docPDFtImage=convert_from_path(url_init)
  jpgurl=url_final+file_name+'.jpg'
  i=1
  for page in docPDFtImage:
    if i == 1:
      page.save(jpgurl, 'JPEG')
    else:
      pass
    i+=1
  return jpgurl

In [5]:
def extractlabel(name):
  name=name[:-8]
  return name

In [6]:
def cargarPDF(file,path,path_ouput):
  #Listas para capturar
  docs = []
  files = []
  label=[]

  #Iteracion para anexar el contenido de los pdf's y las rutas de dichos pdf's
 
  path_file=path+file
  docs.append(DocumentFile.from_pdf(path_file))
  newpathFile=pdfToImage(path_file,path_ouput+(file[:-8]).rstrip()+'/',file[:-4])
  files.append(newpathFile)
  label.append(file[:-8])

  return docs,files,label

In [7]:
def procesarPDFS(listaPDF):
  #Se procesan los resultados del predictor, y se almacenan en una lista
  results=[]

  for j in listaPDF:
    results.append(predictor(j))


  #Se capturan los resultados de la extraccion de texto (formato JSON)
  results_json=[]

  for doc in results:
    results_json.append(doc.export())
    
  #Se realiza la extraccion del texto (words) y coordenadas del texto (boxes)
  words_total=[]
  boxes_total=[]

  for doc in tqdm(results_json):

    words,boxes=text_capture(doc)

    words_total.append(words)
    boxes_total.append(boxes)

  return words_total,boxes_total



In [8]:
path='/content/drive/MyDrive/Colab Notebooks/LayoutMXL/data/datasetRaw/'
listDir=os.listdir(path)
len(listDir)

145

In [10]:
df_list = []
# documentos en español
path_ouput='/content/drive/MyDrive/Colab Notebooks/LayoutMXL/data/train/'
for x in listDir:
  pdfList,files,label=cargarPDF(x,path,path_ouput)
  words_total,boxes_total=procesarPDFS(pdfList)
  #proceso de listado de la ruta(path), con la lista de palabras(words) y coordenadas(boxes)
  #para formatearlo de la manera requerida
  for j in range(len(words_total)):
    df=pd.DataFrame({"label": label[j] ,"path": files[j] ,"words": [words_total[j]], "boxes": [boxes_total[j]]})
    df_list.append(df)

  #df_list[0].iloc[0]["boxes"]

  df_final = pd.concat(df_list, ignore_index=True)

  df_final


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
df_final

,label,path,words,boxes
0,Quimica,/content/drive/MyDrive/Colab Notebooks/LayoutM...,"[Sharon, Eliana, Villamil-Silva',, Lesly, Joha...","[[176, 64, 219, 75], [220, 63, 258, 75], [257,..."
1,Quimica,/content/drive/MyDrive/Colab Notebooks/LayoutM...,"[Sergio, Acevedo',, Liliana, Giraldo', &., Jua...","[[264, 63, 305, 78], [304, 64, 370, 76], [369,..."
2,Quimica,/content/drive/MyDrive/Colab Notebooks/LayoutM...,"[Camila, A., Garcia-, Carreno',, Camila, Cardo...","[[283, 64, 327, 76], [325, 63, 342, 77], [341,..."
3,Quimica,/content/drive/MyDrive/Colab Notebooks/LayoutM...,"[Deisy, Cisneros-Sanchez"",, Alexis, Otero-Calv...","[[145, 63, 182, 78], [181, 63, 301, 77], [299,..."
4,Quimica,/content/drive/MyDrive/Colab Notebooks/LayoutM...,"[Leenin, Flores-Ramos',, Anthony, Ruiz-Soto',,...","[[206, 64, 247, 76], [248, 64, 341, 75], [340,..."
...,...,...,...,...
140,CienciasSalud,/content/drive/MyDrive/Colab Notebooks/LayoutM...,"[REVISIONDETEMA, 2005, Vol., 20, No, 3, Rev, C...","[[87, 49, 231, 63], [615, 50, 648, 65], [656, ..."
141,CienciasSalud,/content/drive/MyDrive/Colab Notebooks/LayoutM...,"[TRABAJO, ORIGINAL, 2004, Vol., 191, No, 41, R...","[[86, 49, 165, 63], [165, 49, 247, 63], [614, ..."
142,CienciasSalud,/content/drive/MyDrive/Colab Notebooks/LayoutM...,"[ARTICULO, ORIGINAL, 2006, Vol., 21, No, 11, R...","[[88, 47, 168, 62], [169, 49, 248, 63], [614, ..."
143,CienciasSalud,/content/drive/MyDrive/Colab Notebooks/LayoutM...,"[Rev, Colomb, Cir., 2022;37:184-93, ntps/d0ion...","[[681, 72, 709, 85], [708, 73, 756, 84], [755,..."


In [12]:
# documentos en español
url_output='/content/drive/MyDrive/Colab Notebooks/LayoutMXL/data/train/dataseTrain.pkl'

In [13]:
df_final.to_pickle(url_output)

In [14]:
prueba=pd.read_pickle(url_output)

In [15]:
prueba

,label,path,words,boxes
0,Quimica,/content/drive/MyDrive/Colab Notebooks/LayoutM...,"[Sharon, Eliana, Villamil-Silva',, Lesly, Joha...","[[176, 64, 219, 75], [220, 63, 258, 75], [257,..."
1,Quimica,/content/drive/MyDrive/Colab Notebooks/LayoutM...,"[Sergio, Acevedo',, Liliana, Giraldo', &., Jua...","[[264, 63, 305, 78], [304, 64, 370, 76], [369,..."
2,Quimica,/content/drive/MyDrive/Colab Notebooks/LayoutM...,"[Camila, A., Garcia-, Carreno',, Camila, Cardo...","[[283, 64, 327, 76], [325, 63, 342, 77], [341,..."
3,Quimica,/content/drive/MyDrive/Colab Notebooks/LayoutM...,"[Deisy, Cisneros-Sanchez"",, Alexis, Otero-Calv...","[[145, 63, 182, 78], [181, 63, 301, 77], [299,..."
4,Quimica,/content/drive/MyDrive/Colab Notebooks/LayoutM...,"[Leenin, Flores-Ramos',, Anthony, Ruiz-Soto',,...","[[206, 64, 247, 76], [248, 64, 341, 75], [340,..."
...,...,...,...,...
140,CienciasSalud,/content/drive/MyDrive/Colab Notebooks/LayoutM...,"[REVISIONDETEMA, 2005, Vol., 20, No, 3, Rev, C...","[[87, 49, 231, 63], [615, 50, 648, 65], [656, ..."
141,CienciasSalud,/content/drive/MyDrive/Colab Notebooks/LayoutM...,"[TRABAJO, ORIGINAL, 2004, Vol., 191, No, 41, R...","[[86, 49, 165, 63], [165, 49, 247, 63], [614, ..."
142,CienciasSalud,/content/drive/MyDrive/Colab Notebooks/LayoutM...,"[ARTICULO, ORIGINAL, 2006, Vol., 21, No, 11, R...","[[88, 47, 168, 62], [169, 49, 248, 63], [614, ..."
143,CienciasSalud,/content/drive/MyDrive/Colab Notebooks/LayoutM...,"[Rev, Colomb, Cir., 2022;37:184-93, ntps/d0ion...","[[681, 72, 709, 85], [708, 73, 756, 84], [755,..."
